In [7]:
import tensorflow as tf
import os
import json
import pandas as pd
import re
import numpy as np
import time
import matplotlib.pyplot as plt
import collections
import random
import requests
import json
from math import sqrt
from PIL import Image
from tqdm.auto import tqdm
import os

In [8]:
covidDataPath='../input/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID'
pneumoniaDataPath='../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Viral Pneumonia'
normalDataPath='../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal'
lungopacityDataPath = '../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Lung_Opacity'

In [9]:
# Lists for access paths
listCovidPaths = []
listNormalPaths = []
listPneumoniaPaths = []
listLOPaths = []

# Get covid images files paths
for root, directories, files in os.walk(covidDataPath):
    for name in files:
        listCovidPaths.append(os.path.join(root, name))
# Get Pneumonia images files paths
for root, directories, files in os.walk(pneumoniaDataPath):
    for name in files:
        listPneumoniaPaths.append(os.path.join(root, name))        
        
# Get normal images files paths
for root, directories, files in os.walk(normalDataPath):
    for name in files:
        listNormalPaths.append(os.path.join(root, name))

# Get lung opacity images files paths
for root, directories, files in os.walk(lungopacityDataPath):
    for name in files:
        listLOPaths.append(os.path.join(root, name))    
    
    
# Shuffle lists for random train / test

random.shuffle(listCovidPaths)
random.shuffle(listNormalPaths)
random.shuffle(listPneumoniaPaths)
random.shuffle(listLOPaths)

In [10]:
len(listCovidPaths),len(listNormalPaths),len(listPneumoniaPaths),len(listLOPaths)

(7232, 20384, 2690, 12024)

In [6]:
len(listCovidPaths),len(listNormalPaths),len(listPneumoniaPaths),len(listLOPaths)

(7232, 20384, 2690, 12024)

In [11]:
# main folder
!mkdir ./Data/

# Train data folders
!mkdir ./Data/Train/

# Valid data folders
!mkdir ./Data/Valid/

# Paths to train folders
pathTrain = './Data/Train/'

# Paths to valid folders
pathValid='./Data/Valid/'


In [12]:
from distutils.file_util import copy_file

In [13]:
"""
Moving files to new folders
"""


# Move covid images files to new folders
for i in range(7232):
    if i < 7232:
        copy_file(listCovidPaths[i], pathTrain)
    else  : 
        copy_file(listCovidPaths[i], pathValid)
        
for i in range(20384):
    if i < 20384:
        copy_file(listNormalPaths[i], pathTrain)   
    else  : 
        copy_file(listNormalPaths[i], pathValid)   
        
for i in range(2690):
    if i < 2690:
        copy_file(listPneumoniaPaths[i], pathTrain)   
    else  : 
        copy_file(listPneumoniaPaths[i], pathValid) 
        
for i in range(12024):
    if i < 12024:
        copy_file(listLOPaths[i], pathTrain)   
    else  : 
        copy_file(listLOPaths[i], pathValid) 

In [14]:
import random

def select_random_covid_strings():
    covid_strings = [
        'Covid is the root cause of this case.',
        'The scenario in question is directly linked to Covid.',
        'The current situation is caused by Covid.',
        'Covid is the driving force behind this matter.',
        'This is an instance where Covid is the main problem.',
        'The patient is diagnosed with Covid.',
        'The cause of the illness is Covid.',
        'Covid is the primary factor in this case.',
        'The issue at hand is due to Covid.',
        'The diagnosis is Covid-related.',
        'Covid is the main concern in this scenario.',
        'The symptoms are consistent with Covid.',
        'The patient is being treated for Covid.',
        'Covid is the main culprit in this case.',
        'The situation is complicated by Covid.',
    ]
    return random.choice(covid_strings)

def select_random_normal_strings():
    normal_strings = [
        'The chest x-ray shows normal lungs in this case.',
        'This scenario involves a normal respiratory system.',
        'The current situation involves healthy lungs.',
        'The patient has normal lungs.',
        'This is an example of a healthy respiratory system being present.',
        'The lungs are functioning normally in this case.',
        'The lungs are clear in the chest x-ray.',
        'The absence of any abnormality in the lungs is noted in this case.',
        'The radiograph shows normal lung fields.',
        'The chest x-ray displays no evidence of disease.',
        'There is no sign of lung pathology in this case.',
        'The lungs are free of any pathology in this case.',
        'The lungs appear normal on imaging studies.',
        'The respiratory system is normal in this case.',

    ]
    return random.choice(normal_strings)

def select_random_pneumonia_strings():
    pneumonia_strings = [
        'Pneumonia is the confirmed diagnosis in this case.',
        'This scenario involves bacterial pneumonia.',
        'The current situation relates to community-acquired pneumonia.',
        'Pneumonia is the underlying factor in this matter.',
        'This is an example of pneumonia being the primary diagnosis.',
        'The patient is diagnosed with pneumonia.',
        'The cause of the illness is pneumonia.',
        'Pneumonia is the primary factor in this case.',
        'The issue at hand is due to pneumonia.',
        'The diagnosis is pneumonia-related.',
        'Pneumonia is the main concern in this scenario.',
        'The symptoms are consistent with pneumonia.',
        'The patient is being treated for pneumonia.',
        'Pneumonia is the main cause of morbidity in this case.',
        'The situation is complicated by pneumonia.',
    ]
    return random.choice(pneumonia_strings)

def select_random_lo_strings():
    lo_strings = [
        'Lung opacity is seen in the chest x-ray in this case.',
        'This scenario involves opacification of the lung fields.',
        'The current situation relates to lung consolidation.',
        'Lung opacity is the underlying factor in this matter.',
        'This is an example of lung consolidation being the primary finding.',
        'The patient has areas of lung opacity.',
        'The cause of the opacity is yet to be determined.',
        'Lung opacity is the primary factor in this case.',
        'The issue at hand is due to lung opacity.',
        'The finding of lung opacity is noted in this case.',
        'Lung opacity is a significant finding on imaging studies.',
        'The opacity is consistent with pneumonia in this case.',
        'The patient is being evaluated for lung opacity.',
        'Lung opacity is a concerning finding in this case.',
        'The opacity is prominent on imaging studies.'
    ]

In [18]:
#for training data
import os
import json

# define the directories and corresponding captions
directories = {
    pathTrain: select_random_covid_strings
}

# create a dictionary of captions
captions = {}
for dir_path, caption in directories.items():
    for root, dirs, files in os.walk(dir_path):
        for file in files:
            if file.endswith('.png'):
                image_path = os.path.join(root, file)
                file_name = os.path.splitext(file)[0]  # extract the file name without extension
                if 'COVID' in file_name.upper():
                    caption_str = select_random_covid_strings()
                elif 'NORMAL' in file_name.upper():
                    caption_str = select_random_normal_strings()
                elif 'PNEUMONIA' in file_name.upper():
                    caption_str = select_random_pneumonia_strings()
                elif 'LO' in file_name.upper():
                    caption_str = select_random_lo_strings()
                else:
                    continue  # skip files that do not match any caption category
                caption_with_info = {
                    'file_name': file,
                    'directory': dir_path,
                    'caption': caption_str
                }
                captions[image_path] = caption_with_info

# save the captions dictionary as a JSON file
with open('/kaggle/working/Data/train_captions.json', 'w') as f:
    json.dump(captions, f)

In [19]:
from IPython.display import FileLink
import shutil
import os
shutil.make_archive('trn', 'zip', '/kaggle/working/Data')
FileLink(r'trn.zip')

/kaggle/working/trn.zip